In [1]:
import string
import numpy as np
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
file=open('t8.shakespeare.txt','r+')
data=file.read()

In [3]:
data=data.split('\n') # removing the \n character 
data=data[253:]
data=' '.join(data) # joining the list of sentences after removing the new line character

In [4]:
def cleaner(data):
    token=data.split()
    table=str.maketrans('','',string.punctuation)
    token=[w.translate(table) for w in token]
    token=[word for word in token if word.isalpha()]
    token=[word.lower() for word in token]
    return token

In [5]:
words=cleaner(data=data)

In [6]:
print(f'Unique words: {len(set(words))}')

Unique words: 27956


In [7]:
seed_length=10+1
sentence=list()
for i in range(seed_length,len(words)):
    sequence=words[i-seed_length:i]
    line=' '.join(sequence)
    sentence.append(line)
    if i >10000:
        break

In [11]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentence)
sequence=tokenizer.texts_to_sequences(sentence)

In [12]:
sequence=np.array(sequence)
X,y=sequence[:,:-1],sequence[:,-1]

In [16]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

2239

In [14]:
y=to_categorical(y,num_classes=vocab_size)

In [17]:
sequence_length=X.shape[1]
sequence_length

10

*Model*

In [18]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=sequence_length))
model.add(LSTM(512,return_sequences=True))
model.add(LSTM(256))
model.add(Dense(125,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 50)            111950    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 512)           1153024   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 125)               32125     
_________________________________________________________________
dense_1 (Dense)              (None, 2239)              282114    
Total params: 2,366,669
Trainable params: 2,366,669
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.fit(X,y,batch_size=512,epochs=100)

Epoch 1/100
20/20 [==============================] - 1s 41ms/step - loss: 0.6650 - accuracy: 0.8284
Epoch 2/100
20/20 [==============================] - 1s 42ms/step - loss: 0.9316 - accuracy: 0.7530
Epoch 3/100
20/20 [==============================] - 1s 41ms/step - loss: 1.5354 - accuracy: 0.6208
Epoch 4/100
20/20 [==============================] - 1s 41ms/step - loss: 2.2692 - accuracy: 0.4989
Epoch 5/100
20/20 [==============================] - 1s 41ms/step - loss: 2.7739 - accuracy: 0.4198
Epoch 6/100
20/20 [==============================] - 1s 41ms/step - loss: 3.0509 - accuracy: 0.3666
Epoch 7/100
20/20 [==============================] - 1s 41ms/step - loss: 2.9589 - accuracy: 0.3682
Epoch 8/100
20/20 [==============================] - 1s 41ms/step - loss: 2.9425 - accuracy: 0.3701
Epoch 9/100
20/20 [==============================] - 1s 41ms/step - loss: 2.6988 - accuracy: 0.3930
Epoch 10/100
20/20 [==============================] - 1s 41ms/step - loss: 2.3494 - accuracy: 0.4494

In [24]:
model.save('model.h5')